## Using InterSystems IRIS Vector Search

### InterSystems IRIS Vector Search: An Overview
InterSystems IRIS Vector Search brings powerful AI and machine learning capabilities directly into your SQL workflows by enabling the storage and querying of high-dimensional vector embeddings within a relational database. Vector search works by comparing embedding vectors—numerical representations of unstructured data like text—to determine semantic similarity, making it ideal for tasks like intelligent search and information retrieval. With InterSystems IRIS, you can store these embeddings using the optimized VECTOR and EMBEDDING data types. The EMBEDDING type streamlines the process by converting text into vectors directly through SQL, without requiring direct interaction with an embedding model. By integrating these capabilities into standard SQL operations, IRIS transforms your relational database into a high-performance hybrid vector database—ready to support next-generation AI applications.

Watch the video below to get an overview of how vector search can power generative AI applications in InterSystems IRIS.

<iframe width="560" height="315" src="https://www.youtube.com/embed/-4SAkjqCpCI?si=_5x94XRFQvnok_U8" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

### Running a Simple Vector Search
In the respository for this workshop, there is a set of medical data that will be used for experimentation. The data set includes 80,000 patient encounters, each with structured and coded medical data. With each encounter, however, is also a generated clinical summary note that provides more context about the patient. This might include things such as their commuting situation, their mood during the encounter, or other information not easily categorized into a structured encounter record.

Run the block of code below to initiate a simple connection to InterSystems IRIS and view a snippet of this data set.

In [ ]:
import os, pandas as pd
from sentence_transformers import SentenceTransformer
from sqlalchemy import create_engine, text

from dotenv import load_dotenv
load_dotenv(override=True)

username = 'SuperUser'
password = 'SYS'
hostname = 'localhost'
port = 1972
namespace = 'IRISAPP'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"
engine = create_engine(CONNECTION_STRING)

df = pd.read_sql("SELECT * FROM GenAI.encounters_vectorized", engine)
df.head()

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
note_search = "Headache"
search_vector = model.encode(note_search, normalize_embeddings=True).tolist() # Convert search phrase into a vector
print(search_vector)

In [ ]:
from sqlalchemy import text

vector_str = ",".join(str(x) for x in search_vector) 

with engine.connect() as conn:
    with conn.begin():
        sql = text("""
            SELECT TOP 3 * 
            FROM GenAI.encounters_vectorized
            ORDER BY VECTOR_DOT_PRODUCT(DESCRIPTION_OBSERVATIONS_Vector, TO_VECTOR(:search_vector)) DESC
        """)
        results = conn.execute(sql, {"search_vector": vector_str}).fetchall()

# Display results
df = pd.DataFrame(results)
df.head()